# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [20]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [21]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [22]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [24]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [25]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
# artist text,first_name text,gender text,item_in_session int,last_name text,length text,level text,location text,sessionid int,song text,userid int
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [26]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    
    CREATE KEYSPACE IF NOT EXISTS project2
    WITH REPLICATION = {
    'class' : 'SimpleStrategy', 'replication_factor' : 1
    }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [27]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Below is the sample of data:

![Sample of Data](image_event_datafile_new.jpg)

#### **We are looking for artists songs were listined during a specific session Id and itemInsession. Looking at the table above, we see that for a given seesion, the itemInSession is unique. Therefore, the combination of both is a good candidate for the Primary key.**

#### **Here are the columns for this query:** 

The columns output by the query are:  

column 1: artist  
column 2: song  
column 3: length

In [28]:
##  Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# Drop table if it exists
query = 'DROP TABLE IF EXISTS music_library_artists_on_session'
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# Creating the table
query = 'CREATE TABLE IF NOT EXISTS music_library_artists_on_session'
query = query + '(artist text, item_in_session int, length float, sessionid int, song text, PRIMARY KEY(sessionid, item_in_session))'

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [29]:
# Inserting data in the table music_library_artists_on_session

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_artists_on_session (artist, item_in_session, length, sessionid, song)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
       
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9] ))

#### Do a SELECT to verify that the data have been inserted into each table

In [31]:
## Add in the SELECT statement to verify the data was entered into the table
## query = Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338 and itemInSession = 4   

query="SELECT artist, length, song FROM music_library_artists_on_session WHERE sessionid = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(f'{row.artist},{row.song},{row.length}')                

Faithless,Music Matters (Mark Knight Dub),495.30731201171875


### **2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userId = 10, sessionId = 182**


Below is the sample of data:

![Sample of Data](image_event_datafile_new.jpg)

#### **Since each session for the same user, it has multiple itemInSession for them, so the combination of the 3 columns (userId, itemInSession and sessionId) is used to uniquely identify a row of the data.  I will use both userId and sessionId as the partition key in the table of music_library_song_by_user_on_session, then add itemInSession  as clustering column used to sort song  in the table to make each Primary Key unique.**

#### **Here are the columns for this query:**

In [32]:
# Creating the tablemusic_library_song_by_user_on_session

query = 'DROP TABLE IF EXISTS music_library_song_by_user_on_session '
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = 'CREATE TABLE IF NOT EXISTS music_library_song_by_user_on_session '
query = query + '(artist text, first_name text, item_in_session int, last_name text, sessionid int, song text, userid int, PRIMARY KEY((userid, sessionid), item_in_session))'

try:
    session.execute(query)
except Exception as e:
    print(e)
    

In [33]:
# Insert data in  the table music_library_song_by_user_on_session

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statemetns into the 'query' variable
        query = "INSERT INTO music_library_song_by_user_on_session (artist, first_name, item_in_session, last_name, sessionid, song, userid)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        # print(line)
        # Assign which column element should be assigned for each column in the INSERT statement.
        
        session.execute(query, (line[0], line[1], int(line[3]), line[4], int(line[8]), line[9], int(line[10]) ) )

In [34]:
# executing the query and and valiate the Data Model
##  Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query="SELECT artist, song, first_name, last_name FROM music_library_song_by_user_on_session WHERE userid = 10 and sessionid = 182 ;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
            
for row in rows:
    print(f'{row.artist},{row.first_name},{row.last_name}, {row.song}') 


Down To The Bone,Sylvie,Cruz, Keep On Keepin' On
Three Drives,Sylvie,Cruz, Greece 2000
Sebastien Tellier,Sylvie,Cruz, Kilometer
Lonnie Gordon,Sylvie,Cruz, Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


### **3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**

Below is the sample of data:

![Sample of Data](image_event_datafile_new.jpg)

#### **The query asks us to list users who have listened to a specific song. I will use song as partition key in music_library_users_by_song, then use userId as clustering column in the table to make each Primary Key unique.**

#### **Here are the columns for this query:**

Tablea Name: music_library_users_by_song 

column 1: first_name  
column 2: last_name  
column 3: song  
column 4: userid
PRIMARY KEY (song, userid)  

The columns output by the query are:  

column 1: first_name  
column 2: last_name  


In [35]:
# Creating the table music_library3

query = 'DROP TABLE IF EXISTS music_library_users_by_song'  
try:
    session.execute(query)

except Exception as e:
    print(e)
    
query = 'CREATE TABLE IF NOT EXISTS music_library_users_by_song'  
query = query +  '(first_name text, last_name text, song text, userid int, PRIMARY KEY(song, userid))'

try:
    session.execute(query)
except Exception as e:
    print(e)
    

In [36]:
#Insert data in the table: music_library3
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library_users_by_song (first_name,last_name, song, userid)"
        query = query + " VALUES(%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[1],line[4],line[9],int(line[10])))

In [38]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query= "SELECT first_name, last_name FROM music_library_users_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    # print(f'{row.first_name},{row.last_name})
    print(row.first_name,row.last_name)                  

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [40]:
## Drop the table before closing out the sessions  

a = 'DROP TABLE IF EXISTS music_library_artists_on_session'
b = 'DROP TABLE IF EXISTS music_library_song_by_user_on_session'
c = 'DROP TABLE IF EXISTS music_library_users_by_song'
queries =[a,b,c]
try:
    for query in queries:
        session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [41]:
session.shutdown()
cluster.shutdown()